# LDA

LDA is a type of topic modeling is used to uncover what a series of documents is really about.

## Latent Dirichlet Allocation
### **Unsupervised Learning Approach**
### Latent -> Existing but not yet developed or known
### Dirichlet -> A type of categorical distribution
### Allocation ->

- Ruduce the dimensionality of text into something managble AND interpertable.
    - Normally dimensionality reduction occurs to reduce computational requirements... but LDA reduces with and end goal of interpretability in mind.
    - Often dimensionality reduction results in uninterpretable dimensions, but this is not the case with LDA.

In [13]:
# Apply NLP to Text Data
# Estimate a Topic Model on Text Data
# Create Visualizations Relating the Topics to Each Label

In [2]:
import pandas as pd

combined = pd.read_csv('data/Combined_News_DJIA.csv')
news = pd.read_csv('data/RedditNews.csv')
market = pd.read_csv('data/upload_DJIA_table.csv')

In [3]:
print(combined.shape)
combined.head(1)

(1989, 27)


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


In [4]:
# Remove 'b'-prefixes
# Applymap -> Like .apply() but Applied to Entire DF
combined = combined.applymap(lambda cell: cell.strip() if type(cell)==str else cell)
combined = combined.applymap(lambda cell: cell.lstrip('b"') if type(cell)==str else cell)
combined.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...,'BREAKING: Musharraf to be impeached.','Russia Today: Columns of troops roll into Sou...,'Russian tanks are moving towards the capital ...,"Afghan children raped with 'impunity,' U.N. of...",'150 Russian tanks have entered South Ossetia ...,"Breaking: Georgia invades South Ossetia, Russi...",The 'enemy combatent' trials are nothing but a...,...,'Georgia Invades South Ossetia - if Russia get...,'Al-Qaeda Faces Islamist Backlash',"'Condoleezza Rice: ""The US would not act to pr...",'This is a busy day: The European Union has a...,"Georgia will withdraw 1,000 soldiers from Iraq...",'Why the Pentagon Thinks Attacking Iran is a B...,'Caucasus in crisis: Georgia invades South Oss...,'Indian shoe manufactory - And again in a ser...,'Visitors Suffering from Mental Illnesses Bann...,"No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,'Why wont America and Nato help us? If they wo...,'Bush puts foot down on Georgian conflict',Jewish Georgian minister: Thanks to Israeli tr...,'Georgian army flees in disarray as Russians a...,"Olympic opening ceremony fireworks 'faked'""",'What were the Mossad with fraudulent New Zeal...,'Russia angered by Israeli military sale to Ge...,'An American citizen living in S.Ossetia blame...,...,'Israel and the US behind the Georgian aggress...,"'""Do not believe TV, neither Russian nor Georg...",'Riots are still going on in Montreal (Canada)...,'China to overtake US as largest manufacturer','War in South Ossetia [PICS]','Israeli Physicians Group Condemns State Torture',' Russia has just beaten the United States ove...,'Perhaps *the* question about the Georgia - Ru...,'Russia is so much better at war',So this is what it's come to: trading sex for ...


In [5]:
# Sample Corpus from Combined DF
# -- Includes Only Top1 Headlines
corpus = combined[['Date', 'Label', 'Top1']]
print(corpus.shape)
corpus.head(3)

(1989, 3)


,Date,Label,Top1
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...
1,2008-08-11,1,'Why wont America and Nato help us? If they wo...
2,2008-08-12,0,'Remember that adorable 9-year-old who sang at...


In [6]:
# Applying NLP

# Input into Gensim Needs to be clean tokens.
# We need to decide what our unit of analysis will be...
# (Lemmas, adjs, keyworkds, nouns, etc.)

# import spacy and load NN
import spacy
nlp = spacy.load('en_core_web_lg')

In [7]:
# Let's try lemmas as a first experiment.

# In practice we would probably need to use generators to read in 
# each document for each iteration... so as to not take up
# too much memory.

# Wrap it in a function
def get_lemmas(text):
   
    lemmas = []
    
    doc = nlp(text)
    
    for token in doc:
        conditions = (token.is_stop == False) & (token.is_punct == False) & (token.is_space == False) & (token.pos_ != 'PRON')
        if conditions:
            lemmas.append(token.lemma_ )
            
    return lemmas

In [8]:
# Example of a Generator

def doc_stream(path):
    for f in ox.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3] == 'txt':
                with open(os.path.join(path, f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(text)
                yield tokens

In [9]:
# Construct Lemmas Column
lemmas = corpus['Top1'].apply(get_lemmas)
corpus.insert(3, 'Lemmas', lemmas)

In [10]:
corpus.head(2)

,Date,Label,Top1,Lemmas
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...,"[Georgia, down, russian, warplane, country, br..."
1,2008-08-11,1,'Why wont America and Nato help us? If they wo...,"[will, not, America, Nato, help, will, not, he..."


## Topic Modeling
- Learn a Vocabulary
- Create a Bag of Words (BoW) Representation of Each Document
- Estimate our LDA model
- Clean up the results
- Add Topic Information Back to DF

In [12]:
import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [17]:
# -> Fit Statement
# -> Estimates a Vocabulary 
id2word = corpora.Dictionary(corpus['Lemmas'])

# -> Transform Statement
# [Vocabulary.bow_method(doc) for each doc]
corpus = [id2word.doc2bow(doc) for doc in corpus['Lemmas']]

# We are using id2word to save on computational resources.
# id2word will require less memory than string data will
# Also, values are stored in a matrix which is also a more coputationally 
# efficient data structure...

In [19]:
# Define Parameters for Our LDA
# --> Has a multi-core implimentation :)

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics = 15,
                   passes=20, # How many times to reallocate the districutions to get a 
                              # better qality model
                              # Analogous to an epoch -> How many times do you fit, 
                              # evaluate, and update parameters of the model
                   workers=4,
                   random_state=812
                  )